# <center> Extensions to Linear Regression

## Outcomes <br>
- Understand the bias-variance tradeoff
- Learn techniques to prevent overfitting
- Utilize Ridge and Lasso regression
- Understand and use AIC and BIC

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline

Generating random data

In [ ]:
X = np.array([np.random.randint(50,100) for i in range(100)])
y = [50*(i-70)**2 + 2.75*i - 5 for i in X] ## quadratic formula
y = [i*(1+0.2*np.random.normal()) for i in y]  ## adding random noise
plt.scatter(X, y)

Linear regression

In [ ]:
model = sm.OLS(y,sm.add_constant(X)).fit()
model.summary()

In [ ]:
plt.scatter(X, y)
X_lin = np.linspace(50,100,100)
plt.plot(X_lin, model.predict(sm.add_constant(X_lin)), color='orange')

Creating a quadratic term

In [ ]:
X_squared = np.array(X)**2 ## square every term in X
X_ = np.stack((X,X_squared), axis=1) ## combine linear and quadratic terms
model = sm.OLS(y,sm.add_constant(X_)).fit() ## linear regression
model.summary() 

In [ ]:
plt.scatter(X, y)
X_lin = np.linspace(50,100,100)
X_lin_ = np.stack((X_lin,X_lin**2), axis=1)
plt.plot(X_lin, model.predict(sm.add_constant(X_lin_)), color='orange')

## Polynomial regression

In [ ]:
X = np.array([np.random.randint(50,100) for i in range(25)])
y = [(i-70)**4 - 25*(i-70)**3 - 50*(i-70)**2 + 2.75*i - 5 for i in X] ## polynomial equation
y = np.array([i*(1+0.25*np.random.normal()) for i in y]) ## adding random noise
plt.scatter(X, y)

In [ ]:
## generates polynomial terms for a single feature
def create_polynomials(data, order):
    features = [data]
    for i in range(2,order+1):
        features.append(data**i)
    return np.stack(features, axis=1)

In [ ]:
X_poly = create_polynomials(X, 3)
X_poly

Using sklearn linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = sm.OLS(y,sm.add_constant(X_poly)).fit() ## linear regression
model.summary() 

In [ ]:
X_lin = np.linspace(50,100,100)
X_lin_poly = create_polynomials(X_lin,3)
plt.scatter(X,y)
plt.plot(X_lin, model.predict(sm.add_constant(X_lin_poly)), color='orange')
plt.legend(['True', 'Pred'])

In [ ]:
models,r2_,mse_ = [],[],[]
for i in range(1,21):
    X_poly = create_polynomials(X, i) ## create polynomials
    model = sm.OLS(y,sm.add_constant(X_poly)).fit(); models.append(model) ## build model
    r2_.append(model.rsquared) ## calculate metrics
    mse_.append(model.mse_model)

In [ ]:
plt.bar([x for x in range(1,21)],r2_);plt.xticks([x for x in range(1,21)]);plt.xlabel('Order'); plt.ylabel('R2');plt.show();plt.bar([x for x in range(1,21)],mse_,color='red');plt.xticks([x for x in range(1,21)]);plt.xlabel('Order'); plt.ylabel('MSE')

In [ ]:
for model,i in zip(models,[x for x in range(1,21)]):
    plt.scatter(X,y) ## plot original data
    X_lin = np.sort(X) ## plot predicted data
    X_lin_poly = create_polynomials(X_lin,i)
    plt.plot(X_lin, model.predict(sm.add_constant(X_lin_poly)), color='orange')
    plt.legend(['True', 'Pred'])
    plt.title("Order: " + str(i))
    plt.show()

In [ ]:
## generating new data
new_X  = np.array([np.random.randint(50,100) for i in range(10)])
new_y = [(i-70)**4 - 25*(i-70)**3 - 50*(i-70)**2 + 2.75*i - 5 for i in new_X] ## polynomial equation
new_y = np.array([i*(1+0.25*np.random.normal()) for i in new_y]) ## adding random noise

In [ ]:
from sklearn.metrics import mean_squared_error
plt.scatter(new_X, new_y)
model = sm.OLS(y,sm.add_constant(create_polynomials(X,21))).fit()
plt.scatter(new_X, model.predict(sm.add_constant(create_polynomials(new_X, 21))))
plt.legend(["True", "Pred"])
print('MSE:', mean_squared_error(new_y, model.predict(sm.add_constant(create_polynomials(new_X, 21)))))

## <center>Overfitting

<center><img src='overfitting.png'>

In [ ]:
plt.scatter(new_X, new_y)
model = sm.OLS(y,sm.add_constant(create_polynomials(X,4))).fit()
plt.scatter(new_X, model.predict(sm.add_constant(create_polynomials(new_X, 4))))
plt.legend(["True", "Pred"])
print('MSE:', mean_squared_error(new_y, model.predict(sm.add_constant(create_polynomials(new_X, 4)))))

## <center> Bias v.s. Variance
<center><img src='bias_variance.png' height=800 width=800>

## <center>Preventing Overfitting

- Train-test split
- Cross-validation
- Adjusted R-squard
- AIC
- BIC

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_friedman1
from sklearn.preprocessing import StandardScaler

In [ ]:
X,y = make_friedman1(100,10,2,random_state=47)
y = y - 20*(X[:,8]*X[:,9])
scaler = StandardScaler()
X = scaler.fit_transform(X)
df=pd.DataFrame(X,columns=['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10'])
df['target']=y
df.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=47)

In [ ]:
model_base = sm.OLS(y_train,sm.add_constant(X_train)).fit()
y_pred_base = model_base.predict(sm.add_constant(X_train))
y_pred_base_test = model_base.predict(sm.add_constant(X_test))
print('Train MSE:', mean_squared_error(y_train,y_pred_base))
print('Test MSE:', mean_squared_error(y_test,y_pred_base_test))

In [ ]:
## adding interaction terms
from sklearn.preprocessing import PolynomialFeatures
X_train_interactions = PolynomialFeatures(interaction_only=True, include_bias=False).fit_transform(X_train)
X_test_interactions = PolynomialFeatures(interaction_only=True, include_bias=False).fit_transform(X_test)
X_train_interactions[0]

In [ ]:
model_interactions = sm.OLS(y_train,sm.add_constant(X_train_interactions)).fit()
y_pred_interactions = model_interactions.predict(sm.add_constant(X_train_interactions))
y_pred_interactions_test = model_interactions.predict(sm.add_constant(X_test_interactions))
print('Train MSE:', mean_squared_error(y_train,y_pred_interactions))
print('Test MSE:', mean_squared_error(y_test,y_pred_interactions_test))

<center>Lower training error, but overfitted

## <center> AIC and BIC

<center>Metrics that seek to maximize model accuracy and minimize model complexity.

<center> Tell you how good your model is at representing the true reality of the data (out of sample)

<center> Used to compare model performance

<center>AIC is optimal when the true model is unknown and not one of candidate models <br>
BIC is optimal when the true model is one of candidate models

In [ ]:
X_interactions = PolynomialFeatures(interaction_only=True, include_bias=False).fit_transform(X)
all_interactions_model = sm.OLS(y,sm.add_constant(X_interactions)).fit()
y_pred = all_interactions_model.predict(sm.add_constant(X_interactions))
print('R2-adj:',all_interactions_model.rsquared_adj)
print('AIC:', all_interactions_model.aic)
print('BIC:', all_interactions_model.bic)

### <center> Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
i_,r2_adj_, aic_, bic_ = [],[],[],[]
aic_2 = []
for i in range(1,X_interactions.shape[1]):
    selector = RFE(LinearRegression(), i, step=1).fit(X_interactions, y)
    y_pred = selector.predict(X_interactions)
    i_.append(i)
    new_X = []
    for i in range(X_interactions.shape[1]):
        if selector.get_support()[i]:
            new_X.append(X_interactions[:,i])
    new_X = np.stack(new_X,axis=1)
    model = sm.OLS(y,sm.add_constant(new_X)).fit()
    aic_.append(model.aic)
    bic_.append(model.bic)
    r2_adj_.append(model.rsquared_adj)

In [ ]:
fig = plt.figure();fig.set_figheight(10);fig.set_figwidth(25);
plt.xticks(i_)
plt.plot(i_,r2_adj_);plt.ylabel('Adj R2');plt.xlabel('n')

In [ ]:
fig, ax_aic = plt.subplots();fig.set_figheight(10);fig.set_figwidth(25);ax_bic = ax_aic.twinx()
ax_aic.plot(i_,aic_,color='red');ax_aic.set_ylabel('AIC');ax_aic.set_xlabel('n');ax_aic.legend(['AIC'])
ax_bic.plot(i_,bic_);ax_bic.set_ylabel('BIC');ax_bic.set_xlabel('n');ax_bic.legend(['BIC'])
plt.xticks(i_);plt.show()

In [ ]:
selector = RFE(LinearRegression(), 19, step=1)
selector = selector.fit(X_interactions, y)
y_pred = selector.predict(X_interactions)
new_X = []
for i in range(X_interactions.shape[1]):
    if selector.get_support()[i]:
        new_X.append(X_interactions[:,i])
new_X = np.stack(new_X,axis=1)
model = sm.OLS(y,sm.add_constant(new_X)).fit()
print('R2-adj:',model.rsquared_adj)
print('AIC:', model.aic)
print('BIC:', model.bic)

## <center> Ridge and Lasso Regression

<center>Regularization techniques used to control model complexity and prevent overfitting.

## <center> Ridge Regression

<center><img src='ridge_cost.png'>

## <center> Lasso Regression

<center><img src='lasso_cost.png'>

Ridge: Includes all features and shrinks coefficients. <br>
Lasso: Includes all features and shrinks coefficients, and also shrinks coefficients to zero, performing feature selection.

Ridge: Better with less features. <br>
Lasso: Better with more features.

Ridge: In the case of correlation, includes both features (splits coefficients between them). <br>
Lasso: In the case of correlation, arbitrarily sets one of the features to zero.

In [ ]:
model_ridge = sm.OLS(y,sm.add_constant(X_interactions))
results = model_ridge.fit_regularized(L1_wt=0)
y_pred_ridge = results.predict(sm.add_constant(X_interactions))
final = sm.regression.linear_model.OLSResults(model_ridge,results.params)
print('R2-adj:',final.rsquared_adj)
print('AIC:', final.aic)
print('BIC:', final.bic)

In [ ]:
results.params

In [ ]:
model_lasso = sm.OLS(y,sm.add_constant(X_interactions))
results = model_lasso.fit_regularized(L1_wt=1)
y_pred_lasso = results.predict(sm.add_constant(X_interactions))
final = sm.regression.linear_model.OLSResults(model_lasso,results.params)
print('R2-adj:',final.rsquared_adj)
print('AIC:', final.aic)
print('BIC:', final.bic)

In [ ]:
results.params

### <center> Comparing the best model 

In [ ]:
selector = RFE(LinearRegression(), 19, step=1).fit(X_train_interactions, y_train)
y_pred = selector.predict(X_train_interactions)
new_X_train,new_X_test = [],[]
for i in range(X_train_interactions.shape[1]):
    if selector.get_support()[i]:
        new_X_train.append(X_train_interactions[:,i])
        new_X_test.append(X_test_interactions[:,i])
new_X_train = np.stack(new_X_train,axis=1)
new_X_test = np.stack(new_X_test,axis=1)
model = sm.OLS(y_train,sm.add_constant(new_X_train)).fit()
y_pred_train = model.predict(sm.add_constant(new_X_train))
y_pred_test = model.predict(sm.add_constant(new_X_test))
print('All interactions');print('----------------');print('Train MSE:', mean_squared_error(y_train,y_pred_interactions));print('Test MSE:', mean_squared_error(y_test,y_pred_interactions_test));print()
print('Reduced model');print('-------------');print('Train MSE:', mean_squared_error(y_train,y_pred_train));print('Test MSE:', mean_squared_error(y_test,y_pred_test))

## <center> Activity

<center>Using the data found in <i>data.csv</i> and the techniques above, find a model of best fit that will perform well on new data.